<a href="https://colab.research.google.com/github/Shivam10816/cs6910_assignment3/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install wandb
import wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {'#': 0, '$': 1, '^': 2}
        self.char2count = {'#': 1, '$': 1, '^': 1}
        self.index2char = {0: '#', 1: '$', 2: '^'}
        self.n_chars = 3  # Count
        self.data = {}
        

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1



class EncodedData:
  def __init__(self,lang):
    self.train_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_train.csv", header = None)
    self.val_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_valid.csv", header = None)
    self.test_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_test.csv", header = None)

    self.input_lang = Lang('eng')
    self.output_lang = Lang(lang)

    # add the words to the respective languages
    for i in range(len(self.train_df)):
        
        self.input_lang.addWord(self.train_df[0][i])
        self.output_lang.addWord(self.train_df[1][i])
    
    max_len_1,max_len_2 = self.max_word_lengths(self.train_df)
    self.train_xE,self.train_yE = self.preprocessing(self.train_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.val_df)
    self.val_xE,self.val_yE = self.preprocessing(self.val_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.test_df)
    self.test_xE,self.test_yE = self.preprocessing(self.test_df,max_len_1,max_len_2)

    print(type(self.train_xE))
    
  def max_word_lengths(self,df):
    max_len_1 = df.iloc[:,0].str.len().max()
    max_len_2 = df.iloc[:,1].str.len().max()
    return max_len_1, max_len_2



  def preprocessing(self,df,max_len_1,max_len_2):
    
    M1 = np.zeros((len(df),max_len_1+1))
    M2 = np.zeros((len(df),max_len_2+2))

    #print(M2.shape)
    
    # Encode column 1
    col1 = df.iloc[:, 0].str.lower().str.split()

    for i in range(len(col1)) :
      word = col1[i][0]
      # print(word," " ,word[0],word[1])
      for j in range(len(word)):
        char =word[j]
        if(self.input_lang.char2index.get(char) is not None):
          M1[i][j]=self.input_lang.char2index.get(char)
        else:
          M1[i][j]=1
    # print(col1)
    #encoded_col1 = np.array([encode(words, dict1, max_len_1) for words in col1])
    
    # Encode column 2
    col2 = df.iloc[:, 1].str.lower().str.split()
    # print((col2[0]))
    for i in range(len(col2)) :
      word = col2[i][0]
      # print(word," " ,word[0],word[1])
      M2[i][0]=2
      for j in range(len(word)):
        char =word[j]
        if(self.output_lang.char2index.get(char) is not None):
          M2[i][j+1]=self.output_lang.char2index.get(char)
        else:
          M2[i][j+1]=1
      
    
    
    return torch.from_numpy(M1),torch.from_numpy(M2)


class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, num_encoders, hidden_size, bidirectional=False, cell='GRU',dropout=0.0):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.bidirectional = bidirectional
        self.cell = cell
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout)

        if self.cell == 'RNN':
            self.encoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional )
        
        elif self.cell == 'GRU':
            self.encoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional)
        else:
            self.encoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_encoders,
                                       bidirectional=bidirectional )
    def init_hidden(self, batch_size):
        num_directions = 2 if self.bidirectional else 1
        if self.cell == 'LSTM':
            return (torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device),
                    torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device))
        else:
          return torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device)
        
    def forward(self, input_seq):
        batch_size = input_seq.size(1)
        
        # Embedding layer
        embedded = self.embedding(input_seq.long())
        
        embedded = self.dropout(embedded)
        # Encoder layers
        encoder_hidden = self.init_hidden(batch_size)

        # print(encoder_hidden.shape)
        if self.cell == 'LSTM':
            (encoder_hidden,encoder_cell) = self.init_hidden(batch_size)
            encoder_outputs, (encoder_hidden, encoder_cell) = self.encoder_rnn(embedded, (encoder_hidden, encoder_cell))
            encoder_hidden = (encoder_hidden,encoder_cell)
        else:
          
          encoder_outputs, encoder_hidden = self.encoder_rnn(embedded, encoder_hidden)
        
        
        return  encoder_outputs,encoder_hidden



class DecoderRNN(nn.Module):
    def __init__(self, output_size, embedding_size, num_encoders, num_decoders, hidden_size, dropout=0.0, cell='GRU'):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.num_decoders = num_decoders
        
        self.cell = cell
        self.embedding_size = embedding_size
        self.dropout = nn.Dropout(dropout)
        self.decoder_fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.embedding = nn.Embedding(output_size, embedding_size)
        

        if self.cell == 'RNN':
            self.decoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_decoders)
        elif self.cell == 'GRU':
            self.decoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_decoders)
                                      
        else :
            self.decoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_decoders)                      
                                     


    def forward(self, input_seq, hidden):
        batch_size = input_seq.shape[0]
        

        
        decoder_input = (input_seq)

        decoder_input = decoder_input.unsqueeze(1)
        
        # Embedding layer
        embedded = self.embedding(decoder_input.long()).view(-1,batch_size,self.embedding_size)
        embedded = self.dropout(embedded)
        decoder_hidden = hidden

        decoder_output , decoder_hidden = self.decoder_rnn(embedded, decoder_hidden)
       
        # Project the output to the output size and apply softmax

        decoder_output = self.softmax(self.decoder_fc(decoder_output))
        
        # Return the output tensor
        return decoder_output,decoder_hidden

def count_exact_matches(pred, target):
    """
    Counts the number of rows in preds tensor that match exactly with each row in y tensor.
    preds: tensor of shape (batch_size, seq_len)
    y: tensor of shape (batch_size, seq_len)
    """
    pred = pred[:, 1:-1] # ignore first and last index of each row
    target = target[:, 1:-1] # ignore first and last index of each row
    count=0;
    for i in range(pred.shape[0]):
      flag = True
      for j in range(target.shape[1]):
        if(target[i][j]!=0 and target[i][j]!=pred[i][j]):
          flag=False
          break;
         
      if(flag):
        count+=1;
    
    return count

def evaluate(X,Y,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type):
    encoder.eval()
    decoder.eval()


    running_loss =0
    correct=0
    total=0
    Data = TensorDataset(X,Y)
    Loader = DataLoader(Data,batch_size=batch_size,shuffle=False) 
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    with torch.no_grad():
      for j,(x,y) in enumerate(Loader):
        loss=0

        x = x.to(device)
        y = y.to(device)

        x = x.transpose(0,1)
        y = y.transpose(0,1)
        encoder_output, encoder_hidden = encoder(x)
        
        if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
          
        
        
        decoder_input =(y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size)
          

        for k in range(1,y.shape[1]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
        pred_output[-1]=decoder_output
        # print(pred_output.shape)
        # print(train_y.shape)
        
        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        y = y.reshape(-1)

        pred_output = pred_output.to(device)
        y =y.to(device)

        loss = loss_fun(pred_output, y.long())
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        

        # Compute the accuracy
        preds = torch.argmax(pred_output, dim=1)

        preds = preds.reshape(batch_size,-1)
        y = y.reshape(batch_size,-1)
        
        # print("------------------Iter next--------------")
        # print(preds[:][0])
        # print(train_y[0])
        
        # print("-------------------------PRED----------------------")
        # print(preds)
        # print("-------------------------Train---------------------")
        # print(train_y)
        correct += count_exact_matches(preds,y)
          # Print the training progress
          
    avg_loss = running_loss / (batch_size * (X.shape[0]/batch_size))
  
    avg_acc = correct / X.shape[0]
    print(' correct[{}/{}] Val_Loss: {:.4f}, val_Accuracy: {:.5f}%'
          .format( correct,X.shape[0], avg_loss, avg_acc*100))
            
   
 

def generateDecoderHidden(cell,num_encoder,num_decoder,encoder_hidden):
    hidden={}
    if(cell=="LSTM"):
      
      hx,state = (encoder_hidden)
      if(num_encoder>num_decoder):
        hx = hx[num_encoder-num_decoder:]
        state = state[num_encoder-num_decoder:]
      elif(num_encoder<num_decoder):
        top = hx[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        hiddenExtra =top.repeat(extra,1,1)
        hx =torch.cat((hx,hiddenExtra),dim=0)
      
        stop = state[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        stateExtra =top.repeat(extra,1,1)
        state =torch.cat((state,stateExtra),dim=0)

      hidden=(hx,state)
    else :

      hidden = encoder_hidden
          
      if(num_encoder>num_decoder):
        hidden = encoder_hidden[num_encoder-num_decoder:]
        
      elif(num_encoder<num_decoder):
        top = encoder_hidden[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        hiddenExtra =top.repeat(extra,1,1)
        hidden =torch.cat((encoder_hidden,hiddenExtra),dim=0)
    return hidden


In [ ]:
print(device)

In [3]:
# get encoded data of perticular language

data = EncodedData("hin")    

from torch.utils.data import TensorDataset






<class 'torch.Tensor'>


In [ ]:
def beam_search(encoder , decoder , test_x,test_y,lang_dict,max_size,batch_size,bidirectional,cell_type,beam_width):
  encoder.eval()
  decoder.eval()

  
  num_encoder = decoder.num_encoders
  num_decoder = decoder.num_decoders
  running_loss =0
  correct=0
  total=0
  Data = TensorDataset(test_x,test_y)
  Loader = DataLoader(Data,batch_size=batch_size,shuffle=False) 
  for j,(test_x,test_y) in enumerate(Loader):

      test_x = test_x.to(device)
      test_x = test_x.transpose(0,1)

      test_y = test_y.to(device)
      test_y = test_y.transpose(0,1)

      # Initialize the output token sequences for each input sequence in the batch
      output_tokens = [[] for _ in range(batch_size)]

      with torch.no_grad():
        encode_output,encoder_hidden = encoder(test_x)

      
      # Initialize the beam search for each input sequence in the batch
      beam = [[] for _ in range(batch_size)]
      for i in range(batch_size):
          beam[i].append((torch.tensor([[lang_dict['^']]], device=device), 0.0, None))

      
    

      if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2
      

      hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
      
      for t in range(max_size):
        new_beam = [[] for _ in range(batch_size)]
        
        # Expand each sequence in the beam for each input sequence in the batch
        for i in range(batch_size):
            if len(beam[i]) == 0:
                continue
            
            for seq, score, hidden in beam[i]:
                # Get the last token in the sequence
                token = seq[-1]
                
                # Decode the next token
                output, hidden = decoder(token,hidden)

                # print(output.shape)
                output = torch.log(output.squeeze(0))
                
                
                # Get the top k tokens and their scores
                topk_scores, topk_tokens = output.topk(beam_width, dim=1)
                topk_scores = topk_scores + score
                
                # print("topk score :-", topk_scores.shape)
                # print("topk tokens :-", topk_tokens.shape)
                if(seq.shape[0]==30):
                  print("Predicted:-", seq[:,0])
                  print("Original :-",test_y[:,i])

                # print("Seq :-", seq.shape)
                # Create new sequences by appending each top k token to the end of the sequence
                for j in range(beam_width):
                    new_seq = torch.cat([seq, topk_tokens[0, j].unsqueeze(0).unsqueeze(0)])
                    new_score = topk_scores[0,j].item()
                    
                    # Add the new sequence to the list of partial sequences for the corresponding input sequence
                    new_beam[i].append((new_seq, new_score, hidden))
        
        # Select the top k sequences with the highest scores for each input sequence in the batch
        for i in range(batch_size):
            beam[i] = sorted(new_beam[i], key=lambda x: x[1], reverse=True)[:beam_width]
            
            # If any sequences in the beam end with the <eos> token, remove them from the beam and add them to the output
            for j in range(len(beam[i])):
                if beam[i][j][0][-1] == lang_dict['#']:
                    output_tokens[i].append([int(idx) for idx in beam[i][j][0].squeeze(0)])
                    beam[i][j] = None
            
            beam[i] = [seq for seq in beam[i] if seq is not None]

          

      

beam_search(encoder , decoder , data.test_xE,data.test_yE,data.output_lang.char2index,30,batch_size,bidirectional,cell_type,beam_width=3) 
  

In [ ]:
learn_rate = 0.001
batch_size = 16
hidden_size = 128
embedding_size = 128
num_encoder = 3
num_decoder = 3
cell_type = 'LSTM'
bidirectional =True
dropout = 0.3
teach_ratio = 0.5
epochs = 1
beam_width=0

def trainRNNandGRU(data,cell_type="RNN",embedding_size=64,num_encoder=2,num_decoders=2,hidden_size=32,batch_size=16,bidirectional=False,dropout=0.0,beam_width=0,epoch=10,learn_rate=0.001,teacher_ratio=0.0):
  input_size = data.input_lang.n_chars
  output_size = data.output_lang.n_chars
  encoder = EncoderRNN( input_size, embedding_size, num_encoder, hidden_size, bidirectional, cell_type,dropout)
  decoder = DecoderRNN(output_size,embedding_size,num_encoder,num_decoder,hidden_size,dropout,cell_type)

  trainData = TensorDataset(data.train_xE, data.train_yE)
  trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True) 



  encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
  decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
  loss_fun=nn.CrossEntropyLoss(reduction="sum")

  encoder.to(device)
  decoder.to(device)

  for i in range(epochs):
  
    running_loss = 0.0
    train_correct = 0

    encoder.train()
    decoder.train()

    for j,(train_x,train_y) in enumerate(trainLoader):
        loss=0
        train_x = train_x.to(device)
        train_y = train_y.to(device)

        train_x = train_x.transpose(0, 1)
        train_y = train_y.transpose(0, 1)
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_output, encoder_hidden = encoder(train_x)
        
        # print(encoder_hidden.shape)
        # # lets move to the decoder
        # if(bidirectional):
        #     split_tensor= torch.split(encoder_output, hidden_size, dim=-1)
        #     encoder_output=torch.add(split_tensor[0],split_tensor[1])/2
        

        
        if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
          
        # Compute the loss and accuracy
        
        decoder_input =(train_y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(train_y.shape[0],batch_size,output_size)
          
        
        for k in range(1,train_y.shape[0]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          # print(indices)
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          
          if((random.random()<teach_ratio)):
            decoder_input=(train_y)[k]
          else:
            decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
        pred_output[-1]=decoder_output
        # print(pred_output.shape)
        # print(train_y.shape)
        # print(pred_output.shape)
        # print(train_y.shape)

        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        train_y = train_y.reshape(-1)

        pred_output = pred_output.to(device)
        train_y = train_y.to(device)
        loss = loss_fun(pred_output, train_y.long())
        
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        # Compute the accuracy
        # print(pred_output.shape)
        preds = torch.argmax(pred_output, dim=1)
        # print("------------------Iter next--------------")
        # print(preds[:][0])
        # print(train_y[0])
        # print(preds.shape)
        # print(train_y.shape)
        # print("-------------------------PRED----------------------")
        # print(preds)
        # print("-------------------------Train---------------------")
        # print(train_y)
        
        # Print the training progress
        if j % 100 == 99:
            avg_loss = running_loss / (batch_size * 100)
            
            print('Epoch [{}/{}], Step [{}/{}], Train_Loss: {:.4f}'
                  .format(i+1, epochs, j+1, len(trainLoader), avg_loss))
            # evaluate(data.val_xE,data.val_yE,encoder,decoder,device,output_size,batch_size,cell_type)
            running_loss = 0.0
            train_correct = 0
            
    evaluate(data.val_xE,data.val_yE,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)

  return encoder ,decoder


encoder,decoder=trainRNNandGRU(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epochs,learn_rate,teach_ratio)

  


/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:775: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


 correct[1618/4096] Val_Loss: 39.4536, val_Accuracy: 39.50195%


In [ ]:


class AttentionDecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_decoder,cell_type, dropout=0.0):
        super(AttentionDecoderRNN, self).__init__()

        # Define the input size, hidden size, output size, and number of layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        # Define the embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        # Define the attention mechanism
        self.attention = nn.Linear(hidden_size + embedding_size, hidden_size)
        self.context_vector = nn.Parameter(torch.zeros(hidden_size))

        # Define the output layer
        self.output_layer = nn.Linear(hidden_size, output_size)

        #define softmax
        self.softmax = nn.LogSoftmax(dim=1)

        if cell_type =="RNN" :
          # Define the RNN layer
          self.decoder_rnn =  nn.RNN(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        elif cell_type == "GRU":
          self.decoder_rnn =  nn.GRU(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        else :
          self.decoder_rnn =  nn.LSTM(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)

    def forward(self, input, hidden, encoder_outputs):
        # Compute the attention scores
        batch_size = input.shape[1]
        seq_len = input.shape[0]
       
        

        return output, hidden, attention_weights

    def init_hidden(self, batch_size, device):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)



In [ ]:
from torch.nn.modules.dropout import Dropout
input_size = data.input_lang.n_chars
output_size = data.output_lang.n_chars
EMBEDDING_SIZE=16
NUM_ENCODERS=2
NUM_DECODERS=2
HIDDEN_SIZE=32
BIDIRECTIONAL=False
BEAM_WIDTH = 1
DROPOUT=0.0
LEARNING_RATE=0.001
PAD_TOKEN=0
NUM_EPOCHS=1
batch_size=2